<a href="https://colab.research.google.com/github/antoinevangorp/DatabasManagementPractice_IBT3205/blob/main/lab_week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install ipython-sql

In [13]:
import sqlite3
import pandas as pd

In [14]:
# Connect to SQLite database ( or create it if it doesn ’t exist )
conn = sqlite3.connect('lab_session.db')
cursor = conn.cursor()

In [15]:
commands = [
  """
    CREATE TABLE Products (
      ProductID INTEGER PRIMARY KEY,
      ProductName TEXT,
      Price REAL
    );
  """,
  """
    CREATE TABLE Customers (
      CustomerID INTEGER PRIMARY KEY,
      CustomerName TEXT,
      ContactNumber TEXT
    );
  """,
  """
    CREATE TABLE Sales (
      SaleID INTEGER PRIMARY KEY,
      ProductID INTEGER,
      CustomerID INTEGER,
      SaleDate DATE,
      SaleAmount REAL,
      FOREIGN KEY (ProductID) REFERENCES Products(ProductID),
      FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );
  """,
  """
    CREATE TABLE Inventories (
      InventoryID INTEGER PRIMARY KEY,
      ProductID INTEGER,
      StockQuantity INTEGER,
      FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    );
  """,
  """
    CREATE TABLE Suppliers (
      SupplierID INTEGER PRIMARY KEY,
      SupplierName TEXT,
      ContactNumber TEXT
    );
  """
]

In [16]:
# Execute the commands to create tables
for command in commands:
  cursor.execute(command)

In [17]:
commands = [
  """
    INSERT INTO Products (ProductName, Price)
    VALUES ('Product A', 20.0), ('Product B', 15.0), ('Product C', 30.0);
  """,
  """
    INSERT INTO Customers (CustomerName, ContactNumber)
    VALUES ('Customer 1', '1234567890'), ('Customer 2', '0987654321'), ('Customer 3', '1122334455');
  """,
  """
    INSERT INTO Sales (ProductID, CustomerID, SaleDate, SaleAmount)
    VALUES (1, 1, '2023-10-01', 40.0), (2, 2, '2023-10-02', 30.0), (3, 3, '2023-10-03', 60.0);
  """,
  """
    INSERT INTO Inventories (ProductID, StockQuantity)
    VALUES (1, 100), (2, 200), (3, 150);
  """,
  """
    INSERT INTO Suppliers (SupplierName, ContactNumber)
    VALUES ('Supplier 1', '1112223333'), ('Supplier 2', '4445556666'), ('Supplier 3', '7778889999');
  """
]

In [18]:
for command in commands:
  cursor.execute(command)

In [19]:
conn.commit()

In [20]:
commands = [
  """
    INSERT INTO Products (ProductName, Price)
    VALUES
      ('Product D', 25.0), ('Product E', 35.0), ('Product F', 28.0),
      ('Product G', 22.0), ('Product H', 19.0), ('Product I', 40.0),
      ('Product J', 29.0), ('Product K', 24.0), ('Product L', 33.0),
      ('Product M', 27.0);
  """,
  """
    INSERT INTO Customers (CustomerName, ContactNumber)
    VALUES
      ('Customer 4', '2223334444'),
      ('Customer 5', '5556667777'),
      ('Customer 6', '8889990000'),
      ('Customer 7', '1213141516'),
      ('Customer 8', '1718192021'),
      ('Customer 9', '2223242526'),
      ('Customer 10', '2728293031'),
      ('Customer 11', '3233343536'),
      ('Customer 12', '3738394041'),
      ('Customer 13', '4243444546');
  """,
  """
    INSERT INTO Sales (ProductID, CustomerID, SaleDate, SaleAmount)
    VALUES
      (4, 4, '2023-10-04', 50.0), (5, 5, '2023-10-05', 70.0), (6, 6, '2023-10-06', 56.0),
      (7, 7, '2023-10-07', 44.0), (8, 8, '2023-10-08', 38.0), (9, 9, '2023-10-09', 80.0),
      (10, 10, '2023-10-10', 58.0), (11, 11, '2023-10-11', 48.0), (12, 12, '2023-10-12', 66.0),
      (13, 13, '2023-10-13', 54.0);
  """,
  """
    INSERT INTO Inventories (ProductID, StockQuantity)
    VALUES
      (4, 120), (5, 220), (6, 180),
      (7, 110), (8, 210), (9, 160),
      (10, 130), (11, 230), (12, 190),
      (13, 140);
  """,
  """
    INSERT INTO Suppliers (SupplierName, ContactNumber)
    VALUES
      ('Supplier 4', '4546474849'),
      ('Supplier 5', '5051525354'),
      ('Supplier 6', '5556575859'),
      ('Supplier 7', '6061626364'),
      ('Supplier 8', '6566676869'),
      ('Supplier 9', '7071727374'),
      ('Supplier 10', '7576777879'),
      ('Supplier 11', '8081828384'),
      ('Supplier 12', '8586878889'),
      ('Supplier 13', '9091929394');
  """
]

In [21]:
for command in commands:
  cursor.execute(command)
conn.commit()

#Window Functions
#Objective: Calculate the running total and ranking of sales for each product.

In [22]:
running_total_query = """
  SELECT
    ProductID,
    SaleDate,
    SaleAmount,
  SUM(SaleAmount) OVER (PARTITION BY ProductID ORDER BY SaleDate) AS
    RunningTotal
  FROM Sales;
"""

In [23]:
running_total_df = pd.read_sql_query(running_total_query, conn)
running_total_df

,ProductID,SaleDate,SaleAmount,RunningTotal
0,1,2023-10-01,40.0,40.0
1,2,2023-10-02,30.0,30.0
2,3,2023-10-03,60.0,60.0
3,4,2023-10-04,50.0,50.0
4,5,2023-10-05,70.0,70.0
5,6,2023-10-06,56.0,56.0
6,7,2023-10-07,44.0,44.0
7,8,2023-10-08,38.0,38.0
8,9,2023-10-09,80.0,80.0
9,10,2023-10-10,58.0,58.0


In [24]:
ranking_query = """
  SELECT
    ProductID,
    SaleDate,
    SaleAmount,
  RANK() OVER (PARTITION BY ProductID ORDER BY SaleAmount DESC) AS
    Rank
  FROM
    Sales;
"""

ranking_df = pd.read_sql_query(ranking_query, conn)
ranking_df

,ProductID,SaleDate,SaleAmount,Rank
0,1,2023-10-01,40.0,1
1,2,2023-10-02,30.0,1
2,3,2023-10-03,60.0,1
3,4,2023-10-04,50.0,1
4,5,2023-10-05,70.0,1
5,6,2023-10-06,56.0,1
6,7,2023-10-07,44.0,1
7,8,2023-10-08,38.0,1
8,9,2023-10-09,80.0,1
9,10,2023-10-10,58.0,1


#Common Table Expressions (CTEs)
### Objective: Create a CTE to calculate the total sales for each product and then query the CTE to find products with total sales greater than a specific amount.

In [25]:
cte_query = """
  WITH TotalSales_CTE AS (
    SELECT
      ProductID,
    SUM(SaleAmount) AS TotalSales
    FROM
      Sales
    GROUP BY
      ProductID
  )
  SELECT
    ProductID,
    TotalSales
  FROM
    TotalSales_CTE
  WHERE
    TotalSales > 40;
"""
cte_df = pd.read_sql_query(cte_query, conn)
cte_df

,ProductID,TotalSales
0,3,60.0
1,4,50.0
2,5,70.0
3,6,56.0
4,7,44.0
5,9,80.0
6,10,58.0
7,11,48.0
8,12,66.0
9,13,54.0


In [26]:
conn.close()